# Description
Multiple sample workflow
## Steps
- Data pre-processing
    - Mapping reads
    - Marking duplicates
    - Recalibrating Base Quality Score
- Joint calling
    - Calling variants per sample
    - Consolidating GVCFs
    - Joint genotyping
- Filtering the Joint callset
- Refining Genotype

### Import module

In [12]:
import os
import pandas as pd

### Create directory

In [13]:
out_dir = "out_dir"
#os.mkdir(out_dir)
out_aln = os.path.join(out_dir,'aln')
#os.mkdir(out_aln)
out_qual = os.path.join(out_dir,'qualification')
#os.mkdir(out_qual)
tmp = os.path.join(out_dir,'tmp')
#os.mkdir(tmp)
out_vcf = os.path.join(out_dir,'gvcf')
#os.mkdir(out_vcf)
out_joint = os.path.join(out_dir,'joint')
#os.mkdir(out_joint)
out_gdb = os.path.join(out_joint,'genomicsdb')
#os.mkdir(out_gdb)
out_fil = os.path.join(out_dir,'filter')
#os.mkdir(out_fil)

### Input

In [14]:
ref_fasta = 'path_ref_fasta'
dbsnp = 'path_dbsnp'
hapmap = 'hapmap_sites.vcf.gz'
omni = '1000G_omni2.5.sites.vcf.gz'
phase1_snp = '1000G_phase1.snps.high_conf.vcf.gz'

known_indel = 'Homo_sapiens_assembly38.known_indels.vcf.gz'
mill_indel = 'Mills_and_1000G_gold_standard.indels.hg38.vcf.gz'

input = pd.read_csv("input.tsv",sep='\t', dtype='str')
input.fillna('', inplace=True)
print (input)

      name         group      read1      read2
0  sample1  readgroup1.1  path1.1.1  path1.1.2
1  sample1  readgroup1.2  path1.2.1  path1.2.2
2  sample1  readgroup1.3  path1.3.1  path1.3.2
3  sample1  readgroup1.4  path1.4.1  path1.4.2
4  sample2  readgroup2.1  path2.1.1  path2.1.2
5  sample2  readgroup2.2  path2.2.1  path2.2.2
6  sample2  readgroup2.3  path2.3.1  path2.3.2
7  sample2  readgroup2.4  path2.4.1  path2.4.2


### Create dictionary

In [15]:
collection={}
collection['samples'] = {}
for i,row in input.iterrows():
    if row['name'] not in collection['samples'].keys():
        collection['samples'][row['name']]={}
    collection['samples'][row['name']][row['group']]={} 
    collection['samples'][row['name']][row['group']]['read1']=row['read1']
    collection['samples'][row['name']][row['group']]['read2']=row['read2']
print(collection)

{'samples': {'sample1': {'readgroup1.1': {'read1': 'path1.1.1', 'read2': 'path1.1.2'}, 'readgroup1.2': {'read1': 'path1.2.1', 'read2': 'path1.2.2'}, 'readgroup1.3': {'read1': 'path1.3.1', 'read2': 'path1.3.2'}, 'readgroup1.4': {'read1': 'path1.4.1', 'read2': 'path1.4.2'}}, 'sample2': {'readgroup2.1': {'read1': 'path2.1.1', 'read2': 'path2.1.2'}, 'readgroup2.2': {'read1': 'path2.2.1', 'read2': 'path2.2.2'}, 'readgroup2.3': {'read1': 'path2.3.1', 'read2': 'path2.3.2'}, 'readgroup2.4': {'read1': 'path2.4.1', 'read2': 'path2.4.2'}}}}


## Each sample 
- bwa mem
- sortbam
- MarkDuplicates
- BaseRecalibrator
- ApplyBQSR
- HaplotypeCaller

In [16]:
list_sample = []
for sample_name in collection['samples']:
    list_sample.append(sample_name)
    sample=collection['samples'][sample_name]
    list_group = []
    for group_name in sample:
        group = sample[group_name]
        list_group.append(group_name)

        #map reads to reference
        group['mappedbam']= os.path.join(out_aln,sample_name+'_'+group_name+'_mapped.bam')
        cmd = rf"""bwa mem -M \
                    -t 20 \
                    -R @RG\tID:{group_name}\tSM:{sample_name}\tLB:lib1\tPL:ILLUMINA \
                    {ref_fasta} \
                    {group['read1']} \
                    {group['read2']} \
                    | samtools view -Shb -o {group['mappedbam']}"""
        print(cmd)
        #os.system(cmd)
        
        #sortbam
        group['sortedbam']= os.path.join(out_aln,sample_name+'_'+group_name+'_sorted.bam')
        cmd = rf"""gatk SortSam \
                -I {group['mappedbam']} \
                -O {group['sortedbam']} \
                -SORT_ORDER coordinate \
                --TMP_DIR {tmp}"""
        print(cmd)
        #os.system(cmd)
    
    #mark duplicate
    sample['markedbam']= os.path.join(out_qual,sample_name+'_marked.bam')
    sample['metrics']= os.path.join(out_qual,sample_name+'_metrics.txt')

    cmd = rf"""gatk MarkDuplicates \
                -O {sample['markedbam']} \
                -M {sample['metrics']} \
                --TMP_DIR {tmp} \
                """
    for gn in list_group:
        cmd += rf"-I {sample[gn]['sortedbam']} "
    print(cmd)
    #os.system(cmd)
    
    #base recalibration
    sample['recaltable']= os.path.join(out_qual,sample_name+'_recal.table')
    cmd = rf"""gatk BaseRecalibrator \
            -I {sample['markedbam']} \
            -R {ref_fasta} \
            --known-sites {dbsnp} \
            -O {sample['recaltable']}"""
    print(cmd)
    #os.system(cmd)
    sample['arrbam']= os.path.join(out_qual,sample_name+'_arr.bam')
    cmd = rf"""gatk ApplyBQSR \
                -R {ref_fasta} \
                -I {sample['markedbam']} \
                --bqsr-recal-file {sample['recaltable']} \
                -O {sample['arrbam']}"""
    print(cmd)
    #os.system(cmd)

    #variant calling
    sample['gvcf']= os.path.join(out_vcf,sample_name+'_g.vcf.gz')
    cmd = rf"""gatk HaplotypeCaller \
                -R {ref_fasta} \
                -I {sample['arrbam']} \
                -O {sample['gvcf']} \
                -ERC GVCF"""
    print(cmd)
    #os.system(cmd)

bwa mem -M \
                    -t 20 \
                    -R @RG\tID:readgroup1.1\tSM:sample1\tLB:lib1\tPL:ILLUMINA \
                    path_ref_fasta \
                    path1.1.1 \
                    path1.1.2 \
                    | samtools view -Shb -o out_dir/aln/sample1_readgroup1.1_mapped.bam
gatk SortSam \
                -I out_dir/aln/sample1_readgroup1.1_mapped.bam \
                -O out_dir/aln/sample1_readgroup1.1_sorted.bam \
                -SORT_ORDER coordinate \
                --TMP_DIR out_dir/tmp
bwa mem -M \
                    -t 20 \
                    -R @RG\tID:readgroup1.2\tSM:sample1\tLB:lib1\tPL:ILLUMINA \
                    path_ref_fasta \
                    path1.2.1 \
                    path1.2.2 \
                    | samtools view -Shb -o out_dir/aln/sample1_readgroup1.2_mapped.bam
gatk SortSam \
                -I out_dir/aln/sample1_readgroup1.2_mapped.bam \
                -O out_dir/aln/sample1_readgroup1.2_sorted.bam \
           

## Consolidating GVCFs

In [17]:
collection['genomicsdb'] = out_gdb
cmd = rf"""gatk GenomicsDBImport \
        --genomicsdb-workspace-path {collection['genomicsdb']} \
        """
for i in list_sample:
    cmd += rf"-V {collection['samples'][i]['gvcf']} "
print(cmd)
#os.system(cmd)

gatk GenomicsDBImport \
        --genomicsdb-workspace-path out_dir/joint/genomicsdb \
        -V out_dir/gvcf/sample1_g.vcf.gz -V out_dir/gvcf/sample2_g.vcf.gz 


## Joint genotyping

In [18]:
collection['jointvcf'] = os.path.join(out_joint,'joint.vcf.gz')
cmd =f"""gatk GenotypeGVCFs
        -R {ref_fasta} \
        -V {collection['genomicsdb']} \
        -O {collection['jointvcf']}"""
print(cmd)
#os.system(cmd) 

gatk GenotypeGVCFs
        -R path_ref_fasta         -V out_dir/joint/genomicsdb         -O out_dir/joint/joint.vcf.gz


## Filtering the Joint Callset 
> Chưa biết cách chọn resource và set arguments


### 1. SNP

In [19]:
collection['snp_recal'] = os.path.join(out_fil,'snp.recal')
collection['snp_tranches'] = os.path.join(out_fil,'snp.tranches')
cmd = rf"""gatk VariantRecalibrator \
                -R {ref_fasta} \
                -V {collection['jointvcf']} \
                --resource:hapmap,known=false,training=true,truth=true,prior=15.0 {hapmap} \
                --resource:omni,known=false,training=true,truth=false,prior=12.0 {omni} \
                --resource:1000G,known=false,training=true,truth=false,prior=10.0 {phase1_snp} \
                --resource:dbsnp,known=true,training=false,truth=false,prior=2.0 {dbsnp} \
                -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
                -mode SNP \
                -O {collection['snp_recal']} \
                --tranches-file {collection['snp_tranches']}"""
print(cmd)
#os.system(cmd) 

gatk VariantRecalibrator \
                -R path_ref_fasta \
                -V out_dir/joint/joint.vcf.gz \
                --resource:hapmap,known=false,training=true,truth=true,prior=15.0 hapmap_sites.vcf.gz \
                --resource:omni,known=false,training=true,truth=false,prior=12.0 1000G_omni2.5.sites.vcf.gz \
                --resource:1000G,known=false,training=true,truth=false,prior=10.0 1000G_phase1.snps.high_conf.vcf.gz \
                --resource:dbsnp,known=true,training=false,truth=false,prior=2.0 path_dbsnp \
                -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
                -mode SNP \
                -O out_dir/filter/snp.recal \
                --tranches-file out_dir/filter/snp.tranches


In [20]:
collection['snp_fil_vcf'] = os.path.join(out_fil,'snp_filtered.vcf.gz')
cmd = rf"""gatk ApplyVQSR \
                -R {ref_fasta} \
                -V {collection['jointvcf']} \
                -O {collection['snp_fil_vcf']} \
                --truth-sensitivity-filter-level 99.9 \
                --tranches-file {collection['snp_tranches']} \
                --recal-file {collection['snp_recal']} \
                -mode SNP"""
print(cmd)
#os.system(cmd) 

gatk ApplyVQSR \
                -R path_ref_fasta \
                -V out_dir/joint/joint.vcf.gz \
                -O out_dir/filter/snp_filtered.vcf.gz \
                --truth-sensitivity-filter-level 99.9 \
                --tranches-file out_dir/filter/snp.tranches \
                --recal-file out_dir/filter/snp.recal \
                -mode SNP


### 2. Indel

In [21]:
collection['indel_recal'] = os.path.join(out_fil,'indel.recal')
collection['indel_tranches'] = os.path.join(out_fil,'indel.tranches')
cmd = rf"""gatk VariantRecalibrator \
                -R {ref_fasta} \
                -V {collection['snp_fil_vcf']} \
                --resource:hapmap,known=false,training=true,truth=true,prior=15.0 {hapmap} \
                --resource:omni,known=false,training=true,truth=false,prior=12.0 {omni} \
                --resource:Mill_1000G,known=false,training=true,truth=false,prior=10.0 {mill_indel} \
                --resource:known_indel,known=true,training=false,truth=false,prior=2.0 {known_indel} \
                -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
                -mode INDEL \
                -O {collection['indel_recal']} \
                --tranches-file {collection['indel_tranches']}"""
print(cmd)
#os.system(cmd) 

gatk VariantRecalibrator \
                -R path_ref_fasta \
                -V out_dir/filter/snp_filtered.vcf.gz \
                --resource:hapmap,known=false,training=true,truth=true,prior=15.0 hapmap_sites.vcf.gz \
                --resource:omni,known=false,training=true,truth=false,prior=12.0 1000G_omni2.5.sites.vcf.gz \
                --resource:Mill_1000G,known=false,training=true,truth=false,prior=10.0 Mills_and_1000G_gold_standard.indels.hg38.vcf.gz \
                --resource:known_indel,known=true,training=false,truth=false,prior=2.0 Homo_sapiens_assembly38.known_indels.vcf.gz \
                -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
                -mode INDEL \
                -O out_dir/filter/indel.recal \
                --tranches-file out_dir/filter/indel.tranches


In [22]:
collection['indel_fil_vcf'] = os.path.join(out_fil,'indel_filtered.vcf.gz')
cmd = rf"""gatk ApplyVQSR \
                -R {ref_fasta} \
                -V {collection['snp_fil_vcf']} \
                -O {collection['indel_fil_vcf']} \
                --truth-sensitivity-filter-level 99.9 \
                --tranches-file {collection['indel_tranches']} \
                --recal-file {collection['indel_recal']} \
                -mode INDEL"""
print(cmd)
#os.system(cmd) 

gatk ApplyVQSR \
                -R path_ref_fasta \
                -V out_dir/filter/snp_filtered.vcf.gz \
                -O out_dir/filter/indel_filtered.vcf.gz \
                --truth-sensitivity-filter-level 99.9 \
                --tranches-file out_dir/filter/indel.tranches \
                --recal-file out_dir/filter/indel.recal \
                -mode INDEL


## Genotype Refinement